### A/334 Parameter Specification
- Autocorrection Delay = 3ms
- Sysmbol interval T = 1/106 seconds
- Marking Frequency Band = 2.5kHz - 5.0kHz

### Sample 
- frequency 48kHz -> 48000 frames/second
- Symbol frames 48000/106 = ca. 453 frames

### Help


In [5]:
import wave
import matplotlib.pyplot as plt
import numpy as np

In [6]:
with wave.open('audio.wav', 'rb') as w:
    framerate = w.getframerate()
    frames = w.getnframes()
    channels = w.getnchannels()
    width = w.getsampwidth()
    print('sampling rate:', framerate, 'Hz')
    print('length:', frames, 'samples')
    print('channels:', channels)
    print('sample width:', width, 'bytes')
    
    data = w.readframes(frames)


sampling rate: 48000 Hz
length: 14880384 samples
channels: 2
sample width: 2 bytes


In [7]:
print(2*2*14880384) #channels * sample width * sample length = 59521536 bytes buffersize
print(len(data))
type(data)

59521536
59521536


bytes

In [8]:
sig = np.frombuffer(data, dtype='<i2').reshape(-1, channels)

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(sig)
plt.show()


ModuleNotFoundError: No module named 'utility'